Specificaţi, implementaţi și testați subalgoritmi pentru problema enuntata. Sa se determine:

Calitatea procesului de recunoastere a textului, atat la nivel de caracter, cat si la nivel de cuvant

a. prin folosirea unei metrici de distanta sau

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time
import os

RED = "\033[31m"
GREEN = "\033[32m"
YELLOW = "\033[33m"
BLUE = "\033[34m"
MAGENTA = "\033[35m"
CYAN = "\033[36m"
WHITE = "\033[37m"
RESET = "\033[0m"

os.environ["VISION_KEY"] ="Your key"
os.environ["VISION_ENDPOINT"] ="Your endpoint"
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [2]:
def levenshteinDistance(str1, str2):
    # programare dinamica :(

    n = len(str1) # lungimea sirului sursa
    m = len(str2) # lungimea sirului target
    dp = [[0 for _ in range(m + 1)] for _ in range(n + 1)]

    # se initializeaza prima linie din matrice cu numere de la 0 la n
    for i in range(n + 1):
        dp[i][0] = i

    # se initializeaza prima coloana din matrice cu numere de la 0 la m
    for j in range(m + 1):
        dp[0][j] = j

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if str1[i - 1] == str2[j - 1]:  # daca valorile sunt egale atunci
                dp[i][j] = dp[i - 1][j - 1] # atunci se ia valoare din stanga sus
            else:
                dp[i][j] = 1 + min(dp[i][j - 1], dp[i - 1][j], dp[i - 1][j - 1]) # se ia valoare din celilele de deasupra si stanga
    return dp[n][m]

In [3]:
files = os.listdir('img')
folder = 'img'

groundTruth = {
                'test1.jpg': ['Acesta este un test', 'Stiu ca nu scriu foarte frumos!'],
                'test2.jpg': ['LIGHT', 'BOX'],
                'test3.png': ['CEVA', 'IN', 'AI'],
               }
OCR_rez = dict()

for file in files:
    img = open(os.path.join(folder,file), "rb")
    read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
    )

    operation_id = read_response.headers['Operation-Location'].split('/')[-1]
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)


    result = []
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                result.append(line.text)

    if file not in OCR_rez:
        OCR_rez[file] = result

print(groundTruth)
print(OCR_rez)


{'test1.jpg': ['Acesta este un test', 'Stiu ca nu scriu foarte frumos!'], 'test2.jpg': ['LIGHT', 'BOX'], 'test3.png': ['CEVA', 'IN', 'AI']}
{'test1.jpg': ['Acesta este un test', 'Itra ca nu salía forete frumos!'], 'test2.jpg': ['LIGHT', 'BOX'], 'test3.png': ['CEVA', 'IN', 'Al']}


In [4]:
print("Calitatea procesului la nivel de caracter:")
for file in OCR_rez:
    print(f"    {CYAN}{file}{RESET}")
    for line1, line2 in zip(groundTruth[file], OCR_rez[file]):
        wordsGroundTruth = line1.split(' ')
        wordsOCR = line2.split(' ')
        for w1, w2 in zip(wordsGroundTruth, wordsOCR):
            print(f"        {GREEN}{w1}{RESET} VS {RED}{w2}{RESET} - diferenta: {levenshteinDistance(w1, w2)}")

print("Calitatea procesului la nivel de cuvant:")
for file in OCR_rez:
    print(f"    {CYAN}{file}{RESET}")
    noOfCorrectWords = 0
    for line1, line2 in zip(groundTruth[file], OCR_rez[file]):
        print(f"        {GREEN}{line1}{RESET}  {RED}{line2}{RESET}")
        for w1, w2 in zip(line1.split(' '), line2.split(' ')):
            if w1 == w2:
                noOfCorrectWords += 1
    print(f"        Numarul de cuvinte corecte este {noOfCorrectWords}")


Calitatea procesului la nivel de caracter:
    test1.jpg
        Acesta VS Acesta - diferenta: 0
        este VS este - diferenta: 0
        un VS un - diferenta: 0
        test VS test - diferenta: 0
        Stiu VS Itra - diferenta: 3
        ca VS ca - diferenta: 0
        nu VS nu - diferenta: 0
        scriu VS salía - diferenta: 4
        foarte VS forete - diferenta: 2
        frumos! VS frumos! - diferenta: 0
    test2.jpg
        LIGHT VS LIGHT - diferenta: 0
        BOX VS BOX - diferenta: 0
    test3.png
        CEVA VS CEVA - diferenta: 0
        IN VS IN - diferenta: 0
        AI VS Al - diferenta: 1
Calitatea procesului la nivel de cuvant:
    test1.jpg
        Acesta este un test  Acesta este un test
        Stiu ca nu scriu foarte frumos!  Itra ca nu salía forete frumos!
        Numarul de cuvinte corecte este 7
    test2.jpg
        LIGHT  LIGHT
        BOX  BOX
        Numarul de cuvinte corecte este 2
    test3.png
        CEVA  CEVA
        IN  IN
        AI  Al
   

b. prin folosirea mai multor metrici de distanta.

In [5]:
def jaccard_similarity(text1, text2):
    set1 = set(text1.lower().split())
    set2 = set(text2.lower().split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    if union == 0:
        return 0
    else:
        return intersection / union

In [6]:
print(f"{YELLOW}Cu Levenshtein{RESET}")

print("Calitatea procesului la nivel de caracter:")
for file in OCR_rez:
    print(f"    {CYAN}{file}{RESET}")
    for line1, line2 in zip(groundTruth[file], OCR_rez[file]):
        wordsGroundTruth = line1.split(' ')
        wordsOCR = line2.split(' ')
        for w1, w2 in zip(wordsGroundTruth, wordsOCR):
            print(f"        {GREEN}{w1}{RESET} VS {RED}{w2}{RESET} - diferenta: {levenshteinDistance(w1, w2)}")

print("Calitatea procesului la nivel de cuvant:")
for file in OCR_rez:
    print(f"    {CYAN}{file}{RESET}")
    noOfCorrectWords = 0
    for line1, line2 in zip(groundTruth[file], OCR_rez[file]):
        print(f"        {GREEN}{line1}{RESET}  {RED}{line2}{RESET}")
        for w1, w2 in zip(line1.split(' '), line2.split(' ')):
            if w1 == w2:
                noOfCorrectWords += 1
    print(f"        Numarul de cuvinte corecte este {noOfCorrectWords}")


print(f"{YELLOW}Cu Jaccard Similarity{RESET}")

print("Calitatea procesului la nivel de caracter:")
for file in OCR_rez:
    print(f"    {CYAN}{file}{RESET}")
    for line1, line2 in zip(groundTruth[file], OCR_rez[file]):
        wordsGroundTruth = line1.split(' ')
        wordsOCR = line2.split(' ')
        for w1, w2 in zip(wordsGroundTruth, wordsOCR):
            print(f"        {GREEN}{w1}{RESET} VS {RED}{w2}{RESET} - diferenta: {jaccard_similarity(w1, w2)}")

print("Calitatea procesului la nivel de cuvant:")
for file in OCR_rez:
    print(f"    {CYAN}{file}{RESET}")
    noOfCorrectWords = 0
    for line1, line2 in zip(groundTruth[file], OCR_rez[file]):
        print(f"        {GREEN}{line1}{RESET}  {RED}{line2}{RESET}")
        for w1, w2 in zip(line1.split(' '), line2.split(' ')):
            if w1 == w2:
                noOfCorrectWords += 1
    print(f"        Numarul de cuvinte corecte este {noOfCorrectWords}")

Cu Levenshtein
Calitatea procesului la nivel de caracter:
    test1.jpg
        Acesta VS Acesta - diferenta: 0
        este VS este - diferenta: 0
        un VS un - diferenta: 0
        test VS test - diferenta: 0
        Stiu VS Itra - diferenta: 3
        ca VS ca - diferenta: 0
        nu VS nu - diferenta: 0
        scriu VS salía - diferenta: 4
        foarte VS forete - diferenta: 2
        frumos! VS frumos! - diferenta: 0
    test2.jpg
        LIGHT VS LIGHT - diferenta: 0
        BOX VS BOX - diferenta: 0
    test3.png
        CEVA VS CEVA - diferenta: 0
        IN VS IN - diferenta: 0
        AI VS Al - diferenta: 1
Calitatea procesului la nivel de cuvant:
    test1.jpg
        Acesta este un test  Acesta este un test
        Stiu ca nu scriu foarte frumos!  Itra ca nu salía forete frumos!
        Numarul de cuvinte corecte este 7
    test2.jpg
        LIGHT  LIGHT
        BOX  BOX
        Numarul de cuvinte corecte este 2
    test3.png
        CEVA  CEVA
        IN  IN
   

La Jaccard se face intersectia/reuniune iar la Levenshtein se face prin matricea de editare (programare dinamica)